### extract_lyrics.ipynb

In [60]:
import os 
import sys
import numpy as np 
import pandas as pd
import requests
import base64
import time
import json
import csv
from requests.exceptions import ConnectionError

from glob import glob

In [41]:
# Import the genius package, and set up the lyrics extractor
import lyricsgenius

genius = lyricsgenius.Genius("88toHQ8Vo6klVNei3gUq6KoOo4ccuhGeVk7JKourVQNxsZM52_SHoZRXJzHpmW31", 
    remove_section_headers=True,
    skip_non_songs=True,
    excluded_terms=["(Remix)", "(Live)"],
    sleep_time=2,
    timeout=10, retries=3)  # space out calls to avoid rate‑limit)

In [55]:
currPath = os.getcwd()
dataPath = os.path.abspath(os.path.join(currPath, os.pardir, 'data/playlist_subset'))
playlists = os.listdir(dataPath); playlists.sort() 
playlists = [x for x in playlists if not x.startswith('.')]; playlists = [x for x in playlists if x.endswith('gz')]

In [57]:
lyrics_dir = os.path.join(dataPath, 'lyrics')
os.makedirs(lyrics_dir, exist_ok=True)

In [ ]:
# For each set of playlists...
for p in playlists[:1]: 
    p_data = pd.read_csv(os.path.join(dataPath, p), compression='gzip')

    # Then, get the unique tracks that are in this dataset
    tracks = np.unique(p_data['track_uri'].values)

    # Then, for each unique track, get the lyrics...
    for t in tracks[300:]: 
        curr_track = p_data[p_data['track_uri'] == t] # current track 
        curr_track_idx = curr_track.index.values # get current index

        # Grab song title and artist
        song_title = curr_track['track_name'].values[0]
        artist = curr_track['artist_name'].values[0]   
        
        # Output file for this playlist
        output_file = os.path.join(lyrics_dir, p.replace('.csv.gz', '_lyrics.csv'))

        # Clean song title if necessary...     
        # Clear out anything after a dash
        dash_idx = [song_title.split().index(x) for x in song_title.split() if '-' in x]
        if dash_idx: 
            song_title = song_title.split()[:dash_idx[0]]
            song_title = (' ').join(song_title)

        # Clear out anything after featuring
        feature_idx = [song_title.split().index(x) for x in song_title.split() if 'featuring' in x]
        if feature_idx: 
            song_title = song_title.split()[:feature_idx[0]]
            song_title = (' ').join(song_title)

        # Clear out anything after duet
        duet_idx = [song_title.split().index(x) for x in song_title.split() if 'duet' in x]
        if duet_idx: 
            song_title = song_title.split()[:duet_idx[0]]
            song_title = (' ').join(song_title)

        # Then, get the song lyrics 
        for attempt in range(3):
            try:
                found_lyrics = genius.search_song(song_title, artist)
                break  # Success
            except ConnectionError as e:
                print(f"Connection error on attempt {attempt + 1}: {e}")
                time.sleep(2 ** attempt)  # exponential backoff
            except Exception as e:
                print(f"Other error: {e}")
                break
        
        # If we were able to find the song lyrics 
        if found_lyrics: 
            # First, clean the string
            lyrics_split = found_lyrics.lyrics.split(' ')

            # Find the index that contains the word "lyrics"
            lyrics_idx = [lyrics_split.index(i) for i in lyrics_split if i.startswith('Lyrics')]

            # Check whether it is an annotation or lyrics 
            check_lyric_str = lyrics_split[lyrics_idx[0]]

            # Create a list that contains a cleaned string of song lyrics
            song_lyric_final = []

            # If there is no song annotation...
            if '\n' in check_lyric_str: 
                song_lyric = lyrics_split[lyrics_idx[0]:] # get the actual lyrics 
                song_lyric[0] = song_lyric[0].split('\n')[1:][0] # get rid of unnecessary characters for first word 
                song_lyric = " ".join(song_lyric) # place lyrics in one string 
                song_lyric = song_lyric.replace('\n', ' ') # get rid of unnecessary characters for our entire lyrics string 
                
            elif not '\n' in check_lyric_str: 
                song_lyric = lyrics_split[lyrics_idx[0]:] # get the actual lyrics 
                song_lyric[0] = check_lyric_str[6:]
                song_lyric = " ".join(song_lyric) # place lyrics in one string 
                song_lyric = song_lyric.replace('\n', ' ') # get rid of unnecessary characters for our entire lyrics string 
                
            # If there is a song annotation...
            else: 
                # Get the actual lyrics 
                lyrics_idx = [lyrics_split.index(i) for i in lyrics_split if '\xa0' in i]
                song_lyric = lyrics_split[lyrics_idx[0]:]

                # Place all the words together in one string 
                song_lyric = " ".join(song_lyric)

                # Then, get rid of certain strings 
                song_lyric = song_lyric.replace('More\xa0\n', '')
                song_lyric = song_lyric.replace('\n', ' ')

            # Place the lyrics into our dataframe accordingly
            for song_idx in curr_track_idx: 
                found_flag = 1

        # If we couldn't find the lyrics....
        else: 
            found_flag = 0
            song_lyric = None

        print(list(tracks).index(t), '/', len(tracks))
        
        # Write this row to the output CSV
        with open(output_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=["track_uri", "track_name", "artist_name", "lyrics", "found_lyrics"])
            writer.writerow({
                "track_uri": t,
                "track_name": song_title,
                "artist_name": artist,
                "lyrics": song_lyric,
                "found_lyrics": found_flag, 
                
            })

        print(f"{list(tracks).index(t)} / {len(tracks)} saved to {os.path.basename(output_file)}")
        time.sleep(5)

    # # Then, save this dataframe accordingly
    # os.makedirs(os.path.join(dataPath, 'lyrics'), exist_ok=True)
    # p_data.to_csv(os.path.join(dataPath, 'lyrics', p.replace('.csv.gz', '_lyrics.csv.gz')), index=False, compression='gzip')

    

Searching for "California Water" by Nipsey Hussle...


/Users/arlenel/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/lyricsgenius/genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


Done.
0 / 111561
0 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Still Got Time" by ZAYN...
Done.
1 / 111561
1 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Te Alabaré" by Alex Campos...
No results found for: 'Te Alabaré Alex Campos'
2 / 111561
2 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Phrases" by John Abercrombie...
No results found for: 'Phrases John Abercrombie'
3 / 111561
3 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Yee" by Deorro...
Done.
4 / 111561
4 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Jasmine Runs Away" by Alan Menken...
Specified song does not contain lyrics. Rejecting.
5 / 111561
5 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Just To See You Smile" by Why Don't We...
Done.
6 / 111561
6 / 111561 saved to playlist_subset_1_lyrics.csv
Searching for "Symphony No.41 in C, K.551" by Wolfgang Amadeus Mozart...
No results found for: 'Symphony No.41 in C, K.551 Wolfgang A

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [63]:
list(tracks).index(t)

300

In [32]:
lyrics_split[lyrics_idx[0]:]

['LyricsClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'your',
 'clap',
 'you',
 'clap',
 'your',
 'hands\nYee\nYee\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'hands\nClap',
 'your',
 'hands\nClap',
 'clap',
 'your',
 'han

In [25]:
lyrics_idx

[2]

In [13]:
lyrics_idx

[]

In [144]:
print(list(tracks).index(t), '/', len(tracks))

0 / 111561


In [160]:
# # Build a dummy for loop to extract lyrics for a particular song

# # First, clean the string
# lyrics_split = found_lyrics.lyrics.split(' ')

# # Find the index that contains the word "lyrics"
# lyrics_idx = [lyrics_split.index(i) for i in lyrics_split if i.startswith('Lyrics')]

# # Check whether it is an annotation or lyrics 
# check_lyric_str = lyrics_split[lyrics_idx[0]]

# # Create a list that contains a cleaned string of song lyrics
# song_lyric_final = []

# # If there is no song annotation...
# if '\n' in check_lyric_str: 
#     song_lyric = lyrics_split[lyrics_idx[0]:] # get the actual lyrics 
#     song_lyric[0] = song_lyric[0].split('\n')[1:][0] # get rid of unnecessary characters for first word 
#     song_lyric = " ".join(song_lyric) # place lyrics in one string 
#     song_lyric = song_lyric.replace('\n', ' ') # get rid of unnecessary characters for our entire lyrics string 
    
# # If there is a song annotation...
# else: 
#     # Get the actual lyrics 
#     lyrics_idx = [lyrics_split.index(i) for i in lyrics_split if '\xa0' in i]
#     song_lyric = lyrics_split[lyrics_idx[0]:]

#     # Place all the words together in one string 
#     song_lyric = " ".join(song_lyric)

#     # Then, get rid of certain strings 
#     song_lyric = song_lyric.replace('More\xa0\n', '')
#     song_lyric = song_lyric.replace('\n', ' ')
